# Data Exploration

This notebook is more or less a sandbox made for me to understand and decide how to prepare the dataset and what target to select.

### Constants and imports

In [1]:
import os
import shutil
import pandas as pd
import pycountry
import pygwalker as pyg
from tqdm.notebook import tqdm

METADATA_DIRECTORY = 'metadata'
JOINED_METADATA_FILE = os.path.join(METADATA_DIRECTORY, 'metadata_joined.csv')
CLASS_EXAMPLES_DIRECTORY = 'class_examples'
EXAMPLES_TO_EXPORT_PER_CLASS = 10

### Loading common dataset

In [2]:
df = pd.read_csv(JOINED_METADATA_FILE, lineterminator='\n')

df.head()

/tmp/ipykernel_1117636/1999751367.py:1: DtypeWarning: Columns (3,4,8) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(JOINED_METADATA_FILE, lineterminator='\n')


,id,title,date,begin_date,end_date,culture,technique,type,department,collection,path
0,cma_96021,Crossbow Bolt,1500s–1600s,NaN,NaN,"Germany, 16th-17th century","wood, leather, steel",Arms and Armor,Medieval Art,MED - Arms & Armor,dataset/raw/cma_96021.jpg
1,cma_384221,"West and South Sides of the Vimana Walls, Grea...",1858,NaN,NaN,"England, 19th century",albumenized salt print from a waxed paper nega...,Photograph,Photography,PH - British 19th Century,dataset/raw/cma_384221.jpg
2,cma_99725,Sword Guard (Tsuba),1615–1868,NaN,NaN,"Japan, Edo period (1615-1868)",iron,Arms and Armor,Japanese Art,Japanese Art,dataset/raw/cma_99725.jpg
3,cma_138286,"The Gotha Missal: Fol. 7r, Text",c. 1375,NaN,NaN,"France, Paris","ink, tempera, and gold on vellum; blind-tooled...",Bound Volume,Medieval Art,MED - Manuscript Illuminations,dataset/raw/cma_138286.jpg
4,cma_102552,Fragment,c. 1900–1910,NaN,NaN,"Italy, Florence, 16th century",glazed terracotta,Sculpture,European Painting and Sculpture,Sculpture,dataset/raw/cma_102552.jpg


### Checking type values

In [3]:
walker = pyg.walk(df)

Box(children=(HTML(value='<div id="ifr-pyg-00061361a6f4b7a5cuzfkTwKRCOjFSPG" style="height: auto">\n    <head>…

### Extracting culture values

In [4]:
country_names = [x.name for x in list(pycountry.countries)] + [x.name for x in list(pycountry.historic_countries)]

map_cache = { # I've iterated a lot on this cache, to minimize false values and unknowns
    'America': 'United States',
    'American': 'United States',
    'Flanders': 'Flanders',
    'Venetian': 'Italy',
    'Florentine': 'Italy',
    'Tuscan': 'Italy',
    'Flemings': 'Flemings',
    'Flemish': 'Flemings',
    'Milanese': 'Italy',
    'Genoese': 'Italy',
    'Eastern India': 'Eastern India',
    'Mughal India': 'Mughal India',
    'Sardinia': 'Italy',
    'Byzantium': 'Byzantium',
    'Byzantium (style of)': 'Byzantium',
    'Bohemia': 'Czechia',
    'North Italy': 'Italy',
    'Ferrarese': 'Italy',
    'Sienese': 'Italy',
    'North coast of Peru': 'Peru',
    'Frankish': 'France',
    'Western India': 'Western India',
    'South German': 'Germany',
    'Italy(?)': 'Italy',
    'Chinese': 'China',
    'Netherlandish': 'Netherlands',
    'Austrian': 'Austria',
    'Bohemian': 'Czechia',
    'Paduan': 'Italy',
    'Veronese': 'Italy',
    'Southern France': 'France',
    'Japanese': 'Japan',
    'Greek': 'Greece',
    'Bolognese': 'Italy',
    'Mantuan': 'Italy',
    'North Italian': 'Italy',
    'Norwegian': 'Norway',
    'Canadian': 'Canada',
    'Belgian': 'Belgium',
    'Nuremberg': 'Germany',
    'Turkey': 'Turkey',
    'Swedish': 'Sweden'
}

def attempt_raw_mapping(x):
    if x in map_cache:
        return map_cache[x]
    for name in country_names:
        if x == name:
            map_cache[x] = name
            return name
    y = x.split(',')[0]
    if y in map_cache:
        map_cache[x] = map_cache[y]
        return map_cache[y]
    for name in country_names:
        if x == name:
            map_cache[x] = name
            map_cache[y] = name
            return name
    return None

def attempt_fuzzy_mapping(x): 
    if x in map_cache:
        return map_cache[x]
    try:
        result = pycountry.countries.search_fuzzy(x)[0]
        if 'country' in result:
            result = result.country.name
        else:
            result = result.name
        map_cache[x] = result
        return result
    except:
        try:
            result = pycountry.historic_countries.search_fuzzy(x)[0]
            if 'country' in result:
                result = result.country.name
            else:
                result = result.name
            map_cache[x] = result
            return result
        except:
            y = x.split(',')[0]
            if y in map_cache:
                map_cache[x] = map_cache[y]
                return map_cache[y]
            try:
                result = pycountry.countries.search_fuzzy(y)[0]
                if 'country' in result:
                    result = result.country.name
                else:
                    result = result.name
                map_cache[x] = result
                return result
            except:
                try:
                    result = pycountry.historic_countries.search_fuzzy(y)[0]
                    if 'country' in result:
                        result = result.country.name
                    else:
                        result = result.name
                    map_cache[x] = result
                    return result
                except:
                    map_cache[x] = 'unknown'
                    return 'unknown'
    map_cache[x] = 'unknown'
    return 'unknown'

def mapping(x):
    x = str(x)
    raw = attempt_raw_mapping(x)
    if raw is not None:
        return raw
    
    fuzzy = attempt_fuzzy_mapping(x)
    return fuzzy

tqdm.pandas()

# df['country'] = df['culture'].progress_map(mapping)

In [5]:
# df['country']

In [6]:
# walker = pyg.walk(df)

In [7]:
#df[df['country'] == 'unknown']

In [8]:
# df[df['country'] == 'unknown'].groupby(['culture'])['culture'].count().reset_index(name='count').sort_values(['count'], ascending=False).head(10)

In [9]:
# list(pycountry.historic_countries)

### Extracting classes: Attempt 2
First attempt didn't cause the network to learn anything after dozen of epochs. This time, I'll pick type.

In [10]:
df = pd.read_csv(JOINED_METADATA_FILE, lineterminator='\n')

df.head()

/tmp/ipykernel_1117636/1999751367.py:1: DtypeWarning: Columns (3,4,8) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(JOINED_METADATA_FILE, lineterminator='\n')


,id,title,date,begin_date,end_date,culture,technique,type,department,collection,path
0,cma_96021,Crossbow Bolt,1500s–1600s,NaN,NaN,"Germany, 16th-17th century","wood, leather, steel",Arms and Armor,Medieval Art,MED - Arms & Armor,dataset/raw/cma_96021.jpg
1,cma_384221,"West and South Sides of the Vimana Walls, Grea...",1858,NaN,NaN,"England, 19th century",albumenized salt print from a waxed paper nega...,Photograph,Photography,PH - British 19th Century,dataset/raw/cma_384221.jpg
2,cma_99725,Sword Guard (Tsuba),1615–1868,NaN,NaN,"Japan, Edo period (1615-1868)",iron,Arms and Armor,Japanese Art,Japanese Art,dataset/raw/cma_99725.jpg
3,cma_138286,"The Gotha Missal: Fol. 7r, Text",c. 1375,NaN,NaN,"France, Paris","ink, tempera, and gold on vellum; blind-tooled...",Bound Volume,Medieval Art,MED - Manuscript Illuminations,dataset/raw/cma_138286.jpg
4,cma_102552,Fragment,c. 1900–1910,NaN,NaN,"Italy, Florence, 16th century",glazed terracotta,Sculpture,European Painting and Sculpture,Sculpture,dataset/raw/cma_102552.jpg


In [11]:
walker = pyg.walk(df)

Box(children=(HTML(value='<div id="ifr-pyg-00061361a72b0116vGIw9X8R6S1uYBco" style="height: auto">\n    <head>…

In [12]:
grouping = {
    'Ceramics': 'Sediment-based wares',
    'Ceramic': 'Sediment-based wares',
    'Stucco': 'Sediment-based wares',
    'Terracottas': 'Terracottas',
    
    'Miniatures': 'Miniature',
    
    'Photographs': 'Photograph',
    'Card photograph': 'Photograph',

    'Basketry': 'Basketry',
    'Vases': 'Vases',
    'Vase': 'Vases',
    'Plasting vase': 'Vases',
    'Spindle Whorl': 'Spindle Whorl',
    'Implements': 'Common Items and Festives',
    'Jar': 'Vessels',
    'Vessels': 'Vessels',
    'Horology': 'Common Items and Festives',
    'Funerary Equipment': 'Funerary Equipment',
    'Funerary Object': 'Funerary Equipment',
    'Tomb Pottery': 'Funerary Equipment',
    'Decorative Art': 'Common Items and Festives',
    'Coins': 'Coins',
    'Coin': 'Coins',
    'Equestrian Equipment': 'Equestrian Equipment',
    
    'Leatherwork': 'Leather',
    
    'Enamels': 'Enamels',
    'Enamel': 'Enamels',
    'Glass': 'Glass and glaze',
    'Stained glass': 'Glass and glaze',
    'Lacquer': 'Lacquer',
    
    'Stone Sculpture': 'Sculpture',
    
    'Jade': 'Gem-based wares',
    'Stone': 'Stone',
    'Gems': 'Gem-based wares',

    'Woodwork': 'Organic-based wares',
    'Wood': 'Organic-based wares',
    'Woodblock': 'Organic-based wares',
    'Furniture and woodwork': 'Organic-based wares',

    'Bone/Ivory': 'Bone/Ivory',
    'Bone': 'Bone/Ivory',
    'Ivory': 'Bone/Ivory',
    'Ivories': 'Bone/Ivory',
    'Ivories and Bone': 'Bone/Ivory',

    'Arms': 'Arms and Armor',
    'Daggers': 'Arms and Armor',
    'Helmets': 'Arms and Armor',
    'Shafted Weapons': 'Arms and Armor',
    'Swords': 'Arms and Armor',
    'Sword Furniture': 'Arms and Armor',
    'Shafted Weapons': 'Arms and Armor',
    'Archery Equipment': 'Arms and Armor',
    'Firearms': 'Arms and Armor',

    'Garment': 'Clothing',
    'Costumes': 'Clothing',

    'Textiles': 'Fabrics and fabric-based manufacture',
    'Textile': 'Fabrics and fabric-based manufacture',
    'Velvet': 'Fabrics and fabric-based manufacture',
    
    'Lace': 'Lace',

    'Aerophone': 'Music instruments',
    'Chordophone': 'Music instruments',
    'Idiophone': 'Music instruments',
    'Musical Instrument': 'Music instruments',
    'Membranophone': 'Music instruments',
    
    'Metalwork': 'Metalurgy',
    'Metal': 'Metalurgy',
    'Bronzes': 'Metalurgy',
    'Gold and Silver': 'Metalurgy',
    'Silver': 'Metalurgy',

    'Drawings': 'Drawings',
    'Drawing': 'Drawings',
    'Drawing and Watercolor': 'Drawings',
    'Illustrated Books': 'Drawings',
    
    'Portrait Miniature': 'Portrait Miniature',
    'Miniature Painting': 'Portrait Miniature',
    
    'Albums': 'Albums',
    'Album': 'Albums',
    
    'Index of American Design': 'Drawings',
    
    'Prints': 'Prints',
    'Print': 'Prints',
    
    'Portfolio': 'Portfolio',
    
    'Paintings': 'Paintings',
    'Painting': 'Paintings',
    'Female portrait': 'Paintings',
    'Male portrait': 'Paintings',
    
    'Calligraphy': 'Calligraphy',

    'Chess Sets': 'Gaming pieces',

    'Book': 'Books',
    'Books': 'Books',
    'Codices': 'Books',
    'Bound Volume': 'Books',
    'Volume': 'Books',

    'Manuscript': 'Manuscripts and illuminations',
    'Manuscripts and Illuminations': 'Manuscripts and illuminations',

    'Jewelry': 'Jewelry',
    'Earring': 'Jewelry',
    'Necklace': 'Jewelry',
    'Ring': 'Jewelry',
    'Bracelet': 'Jewelry',
    
    'Medals and Plaquettes': 'Medals and Plaquettes',
    'Medals': 'Medals and Plaquettes',
    'Netsuke': 'Netsuke',
    'Snuff Bottles': 'Snuff Bottles',
    'Pendant': 'Amulets',
    'Amulets': 'Amulets',
    
    'Fans': 'Fans',
    'Inrō': 'Inrō',

    'Miniature': 'Miniature',
    'Miniature': 'Miniature room',

    'Scarabs': 'Scarab',
    'Scarab': 'Scarab'
}
def get_first_part(x):
    x = str(x)
    if '-' in x:
        return x.split('-')[0]
    return x

def mapping(x):
    x = get_first_part(x)
    if 'Armor' in x:
        return 'Arms and Armor'
    if x in grouping:
        return grouping[x]
    return x

tqdm.pandas()

df['type'] = df['type'].progress_map(mapping)


  0%|          | 0/341710 [00:00<?, ?it/s]

In [13]:
types_to_select = list(df['type'].value_counts()[df['type'].value_counts() > 10].index)
types_to_select.remove('nan')

df = df[df['type'].isin(types_to_select)]

In [14]:
types_to_select

['Prints',
 'Drawings',
 'Photograph',
 'Stereograph',
 'Fabrics and fabric-based manufacture',
 'Paintings',
 'Sediment-based wares',
 'Sculpture',
 'Metalurgy',
 'Vases',
 'Books',
 'Folio',
 'Glass and glaze',
 'Arms and Armor',
 'Coins',
 'Cased object',
 'Vessel',
 'Jewelry',
 'Manuscripts and illuminations',
 'Organic-based wares',
 'Costume and Accessories',
 'Decorative Arts',
 'Lace',
 'Miscellaneous',
 'Common Items and Festives',
 'Gem-based wares',
 'Music instruments',
 'Portfolio',
 'Female figure',
 'Lamp',
 'Embroidery',
 'Gem',
 'Bone/Ivory',
 'Stone',
 'Furniture',
 'Medals and Plaquettes',
 'Enamels',
 'Lacquer',
 'Terracottas',
 'Funerary Equipment',
 'Clothing',
 'Cup',
 'Netsuke',
 'Albums',
 'Religious/Ritual Object',
 'Amulets',
 'Portrait Miniature',
 'Stencils',
 'Basketry',
 'Inrō',
 'Scarab',
 'Equestrian Equipment',
 'Snuff Bottles',
 'Architectural Drawing',
 'Mythological figure',
 'Fans',
 'Detached leaf',
 'Bead',
 'Flask',
 'Architectural fragment',
 '

In [15]:
if not os.path.exists(CLASS_EXAMPLES_DIRECTORY):
    os.mkdir(CLASS_EXAMPLES_DIRECTORY)


In [16]:
for type in types_to_select:
    break
    dir_path = os.path.join(CLASS_EXAMPLES_DIRECTORY, type.replace('/', ', '))
    if not os.path.exists(dir_path):
        os.mkdir(dir_path)
        entries_of_type = df[df['type'] == type]
        df_to_export = entries_of_type.sample(n=EXAMPLES_TO_EXPORT_PER_CLASS)
        for _, row in df_to_export.iterrows():
            shutil.copyfile(row['path'], os.path.join(dir_path, os.path.basename(row['path'])))

In [17]:

pd.set_option('display.max_rows', 300)
df['type'].value_counts()

type
Prints                                          120462
Drawings                                         45623
Photograph                                       32153
Stereograph                                      18236
Fabrics and fabric-based manufacture             14025
Paintings                                        10955
Sediment-based wares                             10874
Sculpture                                         9080
Metalurgy                                         7344
Vases                                             6718
Books                                             5459
Folio                                             5085
Glass and glaze                                   3706
Arms and Armor                                    3454
Coins                                             3123
Cased object                                      1914
Vessel                                            1880
Jewelry                                           1600
Manus